In [ ]:
1.1.1

# modify feat_info['missing_or_unknown']

In [ ]:
#DONE

# Identify missing or unknown data values and convert them to NaNs.

# In this code block we will create a new column in feat_info that will contain
#   the processed missing values
# This is crude but it works and since there are only 85 rows it is fast. 
feat_info['missing'] = feat_info['missing_or_unknown'].copy()
characters = ['', 'X', 'XX']

# iterate through feat_info row by row
for idx in range(feat_info.shape[0]):
    
    # create a list from 'missing' at row idx and remove the brackets
    temp_list  = feat_info.iloc[idx]['missing'][1:-1].split(sep=',')                                
    temp_list2 = []                   
    for c in temp_list:               # need to iterate through the original list 
        if c in characters:           # if 'X' or 'XX', add to new list as is
            temp_list2.append(c)
        else:                         # if not, it's a number and needs
            temp_list2.append(int(c)) #    to be converted to int before added to new list
    temp_list = temp_list2
    
    feat_info.at[idx, 'missing'] = temp_list

#feat_info

In [ ]:
# now we need to apply this to the data set

# algo
#   for each col in azdias:
#      look for entries corresponding to feat_col.at[col, missing]
#      if there are any, replace them with nan

azdias_cols = azdias.columns

for col in azdias_cols:
    temp_feature = azdias[col].copy()
    temp_missing = feat_info.loc[feat_info['attribute'] == col]['missing'].array
    temp_feature[temp_feature.isin(temp_missing[0])] = np.nan
    azdias[col] = temp_feature

azdias.head()

In [ ]:
1.1.2

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.

missing_col_data = {}

for col in azdias.columns:        
    missing_col_data[col] = azdias[col].isnull().sum() 

missing_features = pd.DataFrame.from_dict(missing_col_data, orient='index')
missing_features.rename(columns={0:'# NaNs'}, inplace=True)
missing_features.sort_values(by='# NaNs', ascending=False, inplace=True)
missing_features['% NaNs'] = 100 * missing_features['# NaNs'] / azdias.shape[0]

missing_features['# NaNs'].hist(bins=85)

In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)
# We will use 33$ NaNs as a threshold
col_threshold = 33
features_to_drop = missing_features[missing_features['% NaNs'] > col_threshold].index.tolist()
azdias.drop(features_to_drop, axis=1, inplace=True)
azdias.head()

# This works. rerun the entire book up to this point

In [ ]:
1.1.3

In [ ]:
# How much data is missing in each row of the dataset?


# In this code block we will create a new column in azdias that will contain
#   the number of missing values 
azdias['# NaNs'] = azdias.isnull().sum(axis=1)
azdias['% NaNs'] = 100 * azdias['# NaNs'] / (azdias.shape[1] - 1)
azdias_sorted = azdias.sort_values(by='# NaNs', axis=0, ascending=False)
azdias_sorted

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.

# from the histogram we can see that there are two sets of data pertaining to 
#   NaNs. The groups are split around 5% missing data. We will split accordingly.
#   Note that most of the data has ver little NaNs.

row_nan_threshold = 5
azdias_lower = azdias[azdias['% NaNs'] < row_nan_threshold]
azdias_upper = azdias[azdias['% NaNs'] > row_nan_threshold]

print (azdias_lower.shape, azdias_lower['% NaNs'].max())
print (azdias_upper.shape, azdias_upper['% NaNs'].min())

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.

# we need to identify some columns in each data set that does not have any NaNs.
# there are 24 features that do not have NaNs in either data set.

azdias_upper_notna = azdias_upper[azdias_upper.columns[~azdias_upper.isnull().any()]].columns.values.tolist()
azdias_lower_notna = azdias_lower[azdias_lower.columns[~azdias_lower.isnull().any()]].columns.values.tolist()

azdias_notna = list(set(azdias_upper_notna) & set(azdias_lower_notna)) 
azdias_notna.remove('# NaNs')
azdias_notna.remove('% NaNs')
azdias_lower.drop('# NaNs', axis=1, inplace=True)
azdias_lower.drop('% NaNs', axis=1, inplace=True)

print(azdias_notna[:5])
print(len(azdias_notna))

In [ ]:
1.2

In [ ]:
# we need to get a view of feat_info without the features that we dropped earlier
feat_info_reduced = feat_info.copy()
feat_info_reduced = feat_info_reduced[~feat_info_reduced['attribute'].isin(features_to_drop)]
feat_info_reduced.head()

In [ ]:
1.2.1

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?
categorical = feat_info_reduced[feat_info['type']=='categorical']['attribute'].tolist()

for feature in categorical: 
    values = azdias_lower[feature].unique()
    print(feature, values)
    
# Re-encode categorical variable(s) to be kept in the analysis.

# We'll loop through the categorical features and one encode some of them
# We will make a copy of azdias_lower and use it

azdias_preprocessed = azdias_lower.copy()

for feature in categorical: 
    values = azdias_preprocessed[feature].unique()
    if (values.size > 2) | (type(values[0]) == str):                          # if string or multi-level
        X = pd.get_dummies(azdias_preprocessed[feature], prefix=[feature])    # one hot encode
        azdias_preprocessed[X.columns] = X                                    # concatenate
        azdias_preprocessed.drop(feature, axis=1, inplace=True)               # drop the original feature

print(azdias_preprocessed.columns.size)
print(azdias_preprocessed.columns.tolist())

In [ ]:
1.2.2

In [ ]:
# Let's look at the mixed feature types
mixed = feat_info_reduced[feat_info['type']=='mixed']['attribute'].tolist()

PRAEGENDE_JUGENDJAHRE_map = {0:[0,0],   1:[1,1], 2:[2,1],
                             3:[1,2],   4:[2,2], 5:[1,3],
                             6:[2,3],   7:[2,3], 8:[1,4],
                             9:[2,4],  10:[1,5],11:[2,5],
                             12:[1,5], 13:[2,5],14:[1,6],
                             15:[2,6]
                            }
# We will need to address the NaNs. Lets set them to 0
azdias_preprocessed['PRAEGENDE_JUGENDJAHRE'].fillna(value=0, inplace=True)

# We can create two new features and map them using PRAEGENDE_JUGENDJAHRE_map
azdias_preprocessed['PRAEGENDE_JUGENDJAHRE_movement'] = \
     azdias_preprocessed['PRAEGENDE_JUGENDJAHRE'].map(PRAEGENDE_JUGENDJAHRE_map).map(lambda x: x[0])
azdias_preprocessed['PRAEGENDE_JUGENDJAHRE_decade'] = \
     azdias_preprocessed['PRAEGENDE_JUGENDJAHRE'].map(PRAEGENDE_JUGENDJAHRE_map).map(lambda x: x[1])

# Since we are not able to completely split E and W, we will keep 'PRAEGENDE_JUGENDJAHRE'
#   and not remove it

# Investigate "CAMEO_INTL_2015" and engineer two new variables.

# The range for this is ['51' '24' '43' '54' '22' '14' '13' '15' '34' '55' '12' 
#      '41' '25' '23' '31' '52' '35' '45' '33' '44' '32' nan]
# We have to review the Data_Dictionary to make sense of it.
# -1 is unknown. We will set the NaNs we created earlier to -1.  
# The coding for "wealth" and "life stage from the dictionary is
#    houshold: unknown=-x1 and XX, Wealthy=1x, Properous=2x, Confortable=3x, Less Affluent=4x, Poorer=5x 
#    life stage: unknown=x0, Pre-Fam=x1, Young=x2, Family=x3, Old Family=x4, Elders=x5 
# 
# We can simply split the 10's from the 1's for the new features.

# We need to address the NaNs. Let's set them to a value that we can split in which
#   both the ten's and one's are not used. We'll use 99.
azdias_preprocessed['CAMEO_INTL_2015'].fillna(value=99, inplace=True)

# We now need to split the values. One way to do this is to convert the series in to strings
#   and use .str to slice them. Then we turn them back into ints. 
azdias_preprocessed['CAMEO_INTL_2015']            = azdias_preprocessed['CAMEO_INTL_2015'].astype(str)
azdias_preprocessed['CAMEO_INTL_2015_wealth']     = azdias_preprocessed['CAMEO_INTL_2015'].str[0].astype(int)
azdias_preprocessed['CAMEO_INTL_2015_lifestage']  = azdias_preprocessed['CAMEO_INTL_2015'].str[1].astype(int)
azdias_preprocessed.drop('CAMEO_INTL_2015', axis=1, inplace=True)